# Initial Setup

In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.4 MB/s eta 0:00:00


In [ ]:
# Install required packages
import subprocess
import sys

def install_packages():
    packages = [
        "bitsandbytes>=0.41.1",
        "transformers>=4.35.0",
        "peft>=0.6.0",
        "accelerate>=0.23.0",
        "datasets>=2.14.0",
        "trl>=0.7.2",
        "scipy>=1.11.3",
        "sentencepiece>=0.1.99",
        "protobuf>=4.23.4",
        "einops>=0.7.0"
    ]

    print("Installing required packages...")
    for package in packages:
        print(f"Installing {package}")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    print("All packages installed successfully!")

In [ ]:
# Run package installation
install_packages()

# Now import the required libraries
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    TaskType
)
from trl import SFTTrainer

MODEL_ID = "mistralai/Mistral-7B-v0.1"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


Installing required packages...
Installing bitsandbytes>=0.41.1
Installing transformers>=4.35.0
Installing peft>=0.6.0
Installing accelerate>=0.23.0
Installing datasets>=2.14.0
Installing trl>=0.7.2
Installing scipy>=1.11.3
Installing sentencepiece>=0.1.99
Installing protobuf>=4.23.4
Installing einops>=0.7.0
All packages installed successfully!


In [ ]:
!pip install -U bitsandbytes

In [ ]:
!pip install -U trl

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `LLM_PA4` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `LLM_PA4`


In [ ]:
!pip install gradio flask-cors transformers peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.1 MB/s eta 0:00:00


In [ ]:
!pip install flask_cors

# Loading Base Mistral Model

In [ ]:
import streamlit as st

In [ ]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

In [ ]:
compute_dtype = torch.float16

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

try:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        quantization_config=bnb_config,
        device_map="auto"
    )
except Exception as e:
    print(f"Error loading model with BitsAndBytes: {e}")
    print("Trying to load without quantization...")
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.float16,
        device_map="auto"
    )

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

try:
    model = prepare_model_for_kbit_training(model)
except Exception as e:
    print(f"Error preparing model for kbit training: {e}")
    print("Continuing without prepare_model_for_kbit_training...")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Mist

# UI Gradio

In [ ]:


import gradio as gr
import torch
from transformers import pipeline, AutoTokenizer
from peft import PeftModel

current_adapter = None
final_model = None

# Paths to fine tuned LoRA adapters on Huggingface Hub
ADAPTER_PATHS = {
    "legal": "vaibhav1/lora-mistral-legal",
    "medical": "vaibhav1/lora-mistral-medical",
    "finance": "vaibhav1/lora-mistral-finance"
}

# Load zero classifier
classifier = pipeline("zero-shot-classification", model="knowledgator/comprehend_it-base")

def classify_text(text):
    result = classifier(text, list(ADAPTER_PATHS.keys()))
    return result['labels'][0], float(result['scores'][0])

def generate_response(text, domain, temperature=0.7, top_p=0.9):
    global current_adapter, final_model

    # Input format we followed while fine-tuning
    formatted_input = f"<|im_start|>user\n{text}<|im_end|>\n<|im_start|>assistant\n"

    # Load the appropriate adapter if it's not already loaded
    if current_adapter != domain or final_model is None:
        if final_model is not None:
            del final_model
            torch.cuda.empty_cache()

        final_model = PeftModel.from_pretrained(
            model,  # this is mistral base
            ADAPTER_PATHS[domain],
            device_map="auto",
            torch_dtype=torch.float16
        )
        final_model.eval()
        current_adapter = domain

    tokenizer = AutoTokenizer.from_pretrained(
        model.config._name_or_path,
        trust_remote_code=True
    )

    inputs = tokenizer(formatted_input, return_tensors="pt").to(final_model.device)

    # Define stopping criteria for <|im_end|> token
    eos_token_id = tokenizer.encode("<|im_end|>", add_special_tokens=False)[0]

    with torch.no_grad():
        outputs = final_model.generate(
            **inputs,
            max_new_tokens=250,  # Cap at 250 tokens
            temperature=temperature,
            top_p=top_p,
            eos_token_id=eos_token_id,  # Stop at <|im_end|> token
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=False)  # Keep special tokens to find <|im_end|>

    assistant_prefix = "<|im_start|>assistant\n"
    assistant_end = "<|im_end|>"

    if assistant_prefix in response:
        response_start = response.find(assistant_prefix) + len(assistant_prefix)
        response_end = response.find(assistant_end, response_start)

        if response_end > 0:
            response = response[response_start:response_end]
        else:
            response = response[response_start:]

    return response.strip()

def process_query(query, override_domain, temperature, top_p):
    if override_domain and override_domain != "Auto-detect":
        domain = override_domain
        confidence = 1.0  # Manually selected, so confidence is 100%
    else:
        domain, confidence = classify_text(query)

    response = generate_response(query, domain, temperature, top_p)

    result = f"Domain: {domain.upper()} (Confidence: {confidence:.2f})\n\n"
    result += f"Response:\n{response}"

    return result

demo = gr.Interface(
    fn=process_query,
    inputs=[
        gr.Textbox(lines=5, placeholder="Enter your question here...", label="Question"),
        gr.Dropdown(["Auto-detect"] + list(ADAPTER_PATHS.keys()), value="Auto-detect", label="Override Domain"),
        gr.Slider(minimum=0.1, maximum=1.0, value=0.7, step=0.1, label="Temperature"),
        gr.Slider(minimum=0.1, maximum=1.0, value=0.9, step=0.1, label="Top-p")
    ],
    outputs=gr.Textbox(label="Answer"),
    title="LoRA-Select",
    description="Adaptive Task-Specific Text Generation with Dynamic LoRA Switching",
    article="<div style='text-align: right; margin-top: 20px; opacity: 0.7;'>Vaibhav Thalanki, Akshara Reddy, Lakshmi Vadhanie</div>"
)

demo.launch(share=True)

config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/738M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

Device set to use cuda:0


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://25e341734351f17b0b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


> This notebook needs to be running in current session for the live link to work.

> Made By: Vaibhav Thalanki, Lakshmi Vadhanie, Akshara Reddy